FETCHING TOGETHER API

In [3]:

import getpass
import os
from together import Together

os.environ["TOGETHER_API_KEY"] = getpass.getpass("Enter the api key:")

together_api_key = os.getenv('TOGETHER_API_KEY')

if not together_api_key:
    raise ValueError("API key is not set.")

client = Together(api_key = together_api_key)




In [6]:
client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[{"role": "user", "content": "What are some fun things to do in New York"}],
)

ChatCompletionResponse(id='8982cf342f1fa70b-DEL', object=<ObjectType.ChatCompletion: 'chat.completion'>, created=1719128214, model='meta-llama/Llama-3-8b-chat-hf', choices=[ChatCompletionChoicesData(index=0, logprobs=None, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content="The city that never sleeps! New York City is a vibrant and exciting place to visit, with endless options for entertainment, culture, and adventure. Here are some fun things to do in New York:\n\n**Iconic Landmarks and Attractions**\n\n1. Visit the Statue of Liberty and Ellis Island: Take a ferry to Liberty Island to see the iconic statue up close and visit the Ellis Island Immigration Museum.\n2. Explore the Empire State Building: Enjoy panoramic views of the city from the observation deck on the 86th floor.\n3. Walk across the Brooklyn Bridge: Take in the stunning views of the Manhattan skyline and the East River.\n4. Visit the 9/11 Me

LOADING


In [4]:
import requests
from bs4 import BeautifulSoup, SoupStrainer

url = "https://lilianweng.github.io/posts/2023-03-15-prompt-engineering/"

#it will look like request is coming from a browser
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

response = requests.get(url, headers=headers)

if response.status_code==200:
    strainer = SoupStrainer(class_=("post-title", "post-header", "post-content"))
    
    soup = BeautifulSoup(response.content, "html.parser", parse_only=strainer)
    
    content = soup.get_text()
    content_length = len(content)
    
    print(f"content length: {content_length}")
    
    print(content[:500])
    
else:
    print(f"status_code:{response.status_code}")





content length: 29295


      Prompt Engineering
    
Date: March 15, 2023  |  Estimated Reading Time: 21 min  |  Author: Lilian Weng


Prompt Engineering, also known as In-Context Prompting, refers to methods for how to communicate with LLM to steer its behavior for desired outcomes without updating the model weights. It is an empirical science and the effect of prompt engineering methods can vary a lot among models, thus requiring heavy experimentation and heuristics.
This post only focuses on prompt engineering fo


SPLITTING

In [5]:
def text_split(text, chunk_size = 500, chunk_overlapping = 200):
    start = 0
    chunks = []
    end = chunk_size
    while end <= len(text):
        chunk = text[start:end]
        chunks.append(chunk)
        start += chunk_size - chunk_overlapping
        end = start + chunk_size    
    return chunks 

chunks = text_split(content)

print(f"number of chunks:{len(chunks)}")


number of chunks:96


EMBED & STORE

In [6]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = model.encode(chunks)

import faiss
import numpy as np 

dimension = embeddings.shape[1]
index = faiss.IndexFlatL2(dimension)
index.add(embeddings)

e:\LLMs\cuda\Lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [16]:
client.chat.completions.create(
    model="meta-llama/Llama-3-8b-chat-hf",
    messages=[{"role": "user", "content": "brief me abbout cot"}],
)

ChatCompletionResponse(id='89ab3b5cbe85a737-DEL', object=<ObjectType.ChatCompletion: 'chat.completion'>, created=1719552072, model='meta-llama/Llama-3-8b-chat-hf', choices=[ChatCompletionChoicesData(index=0, logprobs=None, finish_reason=<FinishReason.StopSequence: 'stop'>, message=ChatCompletionMessage(role=<MessageRole.ASSISTANT: 'assistant'>, content='A cot is a type of bed or sleeping platform that is typically designed for infants or young children. Here are some key points about cots:\n\n**Definition:** A cot is a small, low-to-the-ground bed designed for infants or young children, usually up to the age of 3-4 years.\n\n**Design:** Cots are typically rectangular in shape, with a flat surface and a low height (around 30-40 cm or 12-16 inches). They often have a sturdy frame and a soft, padded mattress.\n\n**Features:** Cots may include features such as:\n\n* A removable, washable mattress cover\n* A canopy or mosquito net to keep insects out\n* Storage baskets or shelves for toys a

RETRIEVE

In [8]:
#query = "Explain Few-shot prompting"
query = "Brief me about cot"
query_embd = model.encode([query])

k = 1
distances, indices = index.search(query_embd, k)
results = [chunks[i] for i in indices[0]]


In [9]:
#print([results])

for result in results:
    print(result)
    print("\n---\n")

inal answer. The benefit of CoT is more pronounced for complicated reasoning tasks, while using large models (e.g. with more than 50B parameters). Simple tasks only benefit slightly from CoT prompting.
Types of CoT prompts#
Two main types of CoT prompting:

Few-shot CoT. It is to prompt the model with a few demonstrations, each containing manually written (or model-generated) high-quality reasoning chains.

(All the math reasoning examples are from GSM8k)
Question: Tom and Elizabeth have a compe

---

